In [31]:
from IPython.display import Markdown, display

## Loading Documents

In [5]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter

In [8]:
loader = TextLoader("data/sample.txt", encoding='utf-8')
documents = loader.load()

In [11]:
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

## Embeddings

In [12]:
from langchain.embeddings import GooglePalmEmbeddings

In [14]:
import os
palm_api_key = os.getenv("PALM_API_KEY")

In [15]:
embeddings = GooglePalmEmbeddings(google_api_key=palm_api_key)

## Vector Stores
These allow us to store our text embeddings for querying later. In this notebook we will be using `pinecone`.

In [2]:
from langchain.vectorstores.pinecone import Pinecone

### Connect to Pinecone

In [16]:
import os
import pinecone

# initialize pinecone
pinecone.init(
    api_key=os.getenv("PINECONE_API_KEY_02"),  # find at app.pinecone.io
    environment=os.getenv("PINECONE_ENV"),  # next to api key in console
)


In [19]:

index_name = "langchain-demo"

# First, check if our index already exists. If it doesn't, we create it
if index_name not in pinecone.list_indexes():
    # we create a new index
    pinecone.create_index(name=index_name, metric="cosine", dimension=768)
# The OpenAI embedding model `text-embedding-ada-002 uses 1536 dimensions`
docsearch = Pinecone.from_documents(docs, embeddings, index_name=index_name)

# if you already have an index, you can load it like this
# docsearch = Pinecone.from_existing_index(index_name, embeddings)

query = "What is the point of this text?"
docs = docsearch.similarity_search(query)

In [28]:
index = pinecone.GRPCIndex("langchain-demo")
index.describe_index_stats()

{'dimension': 768,
 'index_fullness': 0.00074,
 'namespaces': {'': {'vector_count': 74}},
 'total_vector_count': 74}

## LLM

In [21]:
from langchain.llms import GooglePalm

In [22]:
llm = GooglePalm(google_api_key=palm_api_key, temperature=0.7)

## RetrievalQA
In this section we explore retrieval using the `stuff` method.

In [20]:
from langchain.chains import RetrievalQA

In [23]:
retriever = docsearch.as_retriever()

In [29]:
qa_stuff = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=retriever, 
    verbose=True
)

In [30]:
query = "What is the point of this text?"

In [33]:
response = qa_stuff.run(query)
display(Markdown(response))



> Entering new RetrievalQA chain...

> Finished chain.


This document provides instructions on how to annotate vehicles, pedestrians, and two-wheelers for validation.

## Other Retrieval Methods

### Map Reduce
Gets a response for each chunk and question pair then uses another llm call to summarize the responses.
Good for recursive document summarization.

- Requires more calls
- Assumes documents (chunks) are independent

### Refine
Iteratively builds answer from past calls (like a cascade).

- Longer answer
- Takes longer (slower)
- At least as many calls as map reduce.

### Map Rerank (Experimental)
Single call for each document returning a score from which we specify the highest score to select.
- Faster